In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random
import collections
import time
import pickle
import re

In [2]:
# In[2]:

hw_dir = '/home/yao/gm/mlds/final'
# hw_dir = '/Users/Apple/Desktop/MLDS/final'
# hw_dir = '/Users/yao/Desktop/trueman/big4/MLDS/final'
test_max_slen = 20
train_softmax_slen = 50
train_max_slen = 70

max_slen = 43
min_slen = 7

batch_size = 1280


# In[3]:

In [3]:
with open('%s/all_pickles'%hw_dir,'rb') as f1:
    [train_data,test_data,word2ix,ix2word,vocab_size] = pickle.load(f1)

In [4]:
# In[4]:

blank_ix = word2ix[' ']
new_data = []
for data in train_data:
    temp = []
    sc = 0
    if(int(data[0]) < 3):
        sc = 0
    elif(int(data[0]) > 3):
        sc = 1
    else:
        continue
    temp.append(sc)
    temp.append([int(x) for x in data[1].split(" ")])
    new_data.append(temp)
train_data = new_data
train_data = [i for i in train_data if len(i[1])<=70 and len(i[1])>=5]

blank_ix = word2ix[' ']
new_data = []
for data in test_data:
    temp = []
    sc = 0
    if(int(data[0]) < 3):
        sc = 0
    elif(int(data[0]) > 3):
        sc = 1
    else:
        continue
    temp.append(sc)
    temp.append([int(x) for x in data[1].split(" ")])
    new_data.append(temp)
test_data = new_data

len(train_data),len(test_data)

(101930, 9264)

In [5]:
# In[5]:

train_data_dict = dict()
for i in train_data:
    len_cmmt = len(i[1])
    if len_cmmt in train_data_dict:
        train_data_dict[len_cmmt].append(i)
    else:
        train_data_dict[len_cmmt] = [i]

# del train_data

In [6]:
# In[5]:

train_data_dict = dict()
for i in test_data:
    len_cmmt = len(i[1])
    if len_cmmt in train_data_dict:
        train_data_dict[len_cmmt].append(i)
    else:
        train_data_dict[len_cmmt] = [i]

# del train_data

In [7]:
amax = [len(i[1]) for i in test_data]

In [8]:
min(amax)

7

In [9]:
len_pos    = max_slen  # 現在在長度多少的file，從長的開始
pad_ix = word2ix['<PAD>']
batch_pos  = 0
def get_batch(batch_size):
    global train_data_dict, len_pos, batch_pos
    batch_size = (batch_size//len_pos)+1 # 越長的句子一個batch就讀越少句，+1是避免0的狀況
    # ====================回傳用的參數=============================
    is_eoi = False  # end of iteration
    is_eof = False
    slen  = len_pos
    # ===========================================================
    # ====================從檔案中讀batch_size句===================
#     try:
    now_datas = train_data_dict[len_pos][batch_pos:batch_pos+batch_size]
#     except:
#         return False,True,None,None,0,0
    batch_pos+=batch_size
    if len(now_datas)<batch_size:
        batch_pos=0
        is_eof = True
    # ===========================================================
    batch_size = len(now_datas) # 因為有可能在檔案的尾巴 讀不滿完整的batch_size 故在此更新batch_size
    # ==================若現在的檔案讀完============================
    if is_eof == True:
        random.shuffle(train_data_dict[len_pos])
        if len_pos == min_slen: # 一個iteration跑完 重新再跑
            len_pos = max_slen
            is_eoi = True
        else:                      # 現在這長度的file完了 開始讀len_pos-1的檔案
            while len_pos >= min_slen:
                len_pos -= 1
                if len_pos in train_data_dict:
                    break
            
            
        if batch_size == 0: # 剛好讀到尾巴了
            return is_eoi,is_eof,None,None,slen,batch_size
    # ============================================================
        
    score = np.array([i[0] for i in now_datas])
    cmmt = np.array([i[1] for i in now_datas])
    return is_eoi,is_eof,score,cmmt,slen,batch_size

In [10]:
# In[9]:

lr = 0.001                   
training_iters = 100      

n_vocab_size = vocab_size
n_filter = 64
n_filter_size = 4
n_classes = 2
n_max_pool = 2
n_hidden_units = 512
n_embed_size = 300
n_batch_size = 5000
rnn_layer_num = 5

In [11]:
# In[10]:

tf.reset_default_graph()

In [12]:
# In[11]:

word_embed = tf.Variable(tf.random_uniform([vocab_size,n_embed_size]))
y_embed = tf.constant(np.eye(n_classes))

In [13]:
# # CNN

# In[12]:

# n_sentence_length = tf.placeholder(tf.int32,[1])
x = tf.placeholder(tf.int32, [None, None])
# [2,4,3]
y = tf.placeholder(tf.int32, [None,])
ph_batch_size = tf.placeholder(tf.int32,[])
ph_sen_len = tf.placeholder(tf.int32,[])

In [14]:
sen_index = x
x_emb = tf.nn.embedding_lookup(word_embed,sen_index)
y_index = y
y_emb = tf.nn.embedding_lookup(y_embed,y_index)

In [15]:
# In[17]:

inW = tf.Variable(tf.random_normal([n_embed_size, n_hidden_units]))
outW = tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
inB = tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ]))
outB = tf.Variable(tf.constant(0.1, shape=[n_classes, ]))

In [16]:
n_steps = ph_sen_len
RNN_input1 = tf.reshape(x_emb,[-1,n_embed_size])
X_in = tf.matmul(RNN_input1, inW) + inB
X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])
X_in = tf.nn.relu(X_in)
lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0)
mul_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell]*rnn_layer_num)
outputs, final_state = tf.nn.dynamic_rnn(mul_lstm, X_in,dtype=tf.float32)
outputs = tf.transpose(outputs, [1, 0, 2])[-1]
outputs1 = tf.reshape(outputs,[-1,n_hidden_units])
pred = tf.matmul(outputs1, outW) + outB
pred_argmax = tf.argmax(pred, 1)

In [17]:
# sin_lstm = tf.contrib.rnn.LSTMCell(lstm_dim, forget_bias=1.0)


In [18]:
# In[19]:

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_emb, logits=pred))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_emb, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))    

In [19]:
# In[20]:

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# a = sess.run(word_embed)
# print(a)
# In[ ]:

In [21]:
saver = tf.train.Saver()
for i in range(training_iters):
    is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)
    batch_now = 0
    while(not is_eoi):
        batch_now += 1
        if batch_size == 0:
            is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)
            continue
        inp = {x:cmmt,y:score,ph_batch_size:batch_size,ph_sen_len:slen}
        _,c,a = sess.run([train_op,cost,accuracy],inp)
#         b = sess.run(RNN_input1,inp)
#         c = sess.run(inW,inp)
#         input()
        print("iter:%d batch:%d loss:%f accuracy:%f %d"%(i,batch_now,c,a,slen))
        # if(batch_now % 10 == 0):
        #     loss,acc = sess.run([cost,accuracy],inp)
        #     print("iter:%d batch:%d loss:%f accuracy:%f"%(i,batch_now,loss,acc))`
        if (batch_now % 100 == 0):
            save_path = saver.save(sess,"./RNN1_model.bin")
        is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)

iter:0 batch:1 loss:0.806436 accuracy:0.299145 43
iter:0 batch:2 loss:13.531912 accuracy:0.786325 43
iter:0 batch:3 loss:23.922825 accuracy:0.290598 43
iter:0 batch:4 loss:12.259099 accuracy:0.735043 43
iter:0 batch:5 loss:1.903592 accuracy:0.324786 43
iter:0 batch:6 loss:5.122417 accuracy:0.803419 43
iter:0 batch:7 loss:8.382551 accuracy:0.658120 43
iter:0 batch:8 loss:2.192862 accuracy:0.692308 43
iter:0 batch:9 loss:11.926012 accuracy:0.264957 43
iter:0 batch:10 loss:4.687327 accuracy:0.341880 43
iter:0 batch:11 loss:1.678669 accuracy:0.743590 43
iter:0 batch:12 loss:3.768550 accuracy:0.684210 43
iter:0 batch:13 loss:1.943956 accuracy:0.841667 42
iter:0 batch:14 loss:2.361493 accuracy:0.783333 42
iter:0 batch:15 loss:2.439878 accuracy:0.700000 42
iter:0 batch:16 loss:1.227723 accuracy:0.691667 42
iter:0 batch:17 loss:1.519044 accuracy:0.158333 42
iter:0 batch:18 loss:1.304616 accuracy:0.241667 42
iter:0 batch:19 loss:0.569487 accuracy:0.750000 42
iter:0 batch:20 loss:0.934285 accura

KeyboardInterrupt: 

In [18]:
c.shape

(300, 512)

In [20]:
sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess,'%s/RNN2_model.bin'%hw_dir)

In [21]:
ans_list = []
is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)
batch_now = 0
while(not is_eoi):
    batch_now += 1
    if batch_size == 0:
        is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)
        print("fuck you")
        continue
    inp = {x:cmmt,y:score,ph_batch_size:batch_size,ph_sen_len:slen}
    ans = sess.run(pred_argmax,inp)
    ans_list.append([cmmt,score,ans])
    is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)

In [22]:
ans_list2 = []
for ans in ans_list:
    bsize = ans[0].shape[0]
    ans[0] = ans[0].tolist()
    ans[1] = ans[1].tolist()
    ans[2] = ans[2].tolist()
    for i in range(bsize):
        cmmt = ''.join([ix2word[j] for j in ans[0][i]])
        ans_list2.append([cmmt,ans[1][i],ans[2][i]])

In [23]:
yes = 0
yes_no = 0
for i in ans_list2:
    if i[1]==i[2]:
        yes+=1
    yes_no +=1

In [24]:
yes/float(yes_no)

0.927537796976242

In [25]:
different_RNN = [ans for ans in ans_list2]

In [26]:
with open('%s/output_diff_RNN.bin'%hw_dir,'wb') as f1:
    pickle.dump(different_RNN,f1)